In [1]:
# prvi pristop do meritev iz MIC IJS, Amoniak 2018
# eksperimenti Sabine Markelj,
# Analiza razmerja vodikovih izotopov v amoniaku

import scipy as sp

import scipy as sp
import os, glob
import importer_quadera

from rga_data_handling import Class as rga
from rga_data_handling import RGA_class_plotter as plot
from rga_data_handling import fit_many_times as fmt

import matplotlib.pyplot as plt


local_path = os.environ['LOCAL_DATA_PATH']

maindir = os.path.join(local_path, 'IJS', 'Sabina','Ammonia_2018')
dirname = os.path.join(maindir, 'ASC_data')
output_path = os.path.join(maindir, 'output')
fajli = glob.glob(os.path.join(dirname, "*.asc"))

calib_fn = 'Prisma_IJS_MIC_2018.txt'
calib_path = os.path.join(maindir, calib_fn)


def load_trace(filename, add=None, interval=None):
    err, tag, c_raw, columns = importer_quadera.read_quadera_MID2(filename)
    ime = os.path.basename(filename)[34:][:-4]
    filename_stripped = os.path.split(filename)[-1]
    datetag = filename_stripped.split(' ')[0]
    if add != None:
        ime = "%s_%s" %(ime, add)
    print "%s %s" %(datetag, ime)
    tag['title'] = ime
    if err != 0:
        print "skipping"
        return [], "", ""
    savename = os.path.join(output_path, "%s.pk" %ime)
    if interval != None:
        mask = (columns['time'] >= interval[0]) & (columns['time'] <= interval[1])
        try:
            for key in columns.keys():
                columns[key] = columns[key][mask]
        except:
            print 'Something wrong with interval'
    trace = rga.Trace(columns, tag)
    trace.set_timecol('time')
    trace.replace_CP(calib_path)
    plot_name = os.path.join(output_path, "%s.png" %ime)
    return trace, savename, plot_name

In [ ]:
# Nekaksen kataog meritev
# 1 - 8.5., mid 1 (idx 3), 0 - 1200 s, TDS NH3;
#     8.5. mid 2 (idx 4) 3400 - 4500 s ponoven TDS NH3, vmes mogoce exposure?
# 2 - 8.5., mid 3 (idx 5), 0 - 1500 s exposure NH3 D2, 4400 - 6000 s TDS
# 3 - 8.5., mid 4 ?? (idx ?), p & T variation during NH3 & D2 exposure
# 4 - 

In [4]:
# 1 - kalibracija za amoniak

idx = 4
tr, sv_fn, pl_fn = load_trace(fajli[idx])

20180508 1802 mid2


In [10]:
# kalibracija za amoniak
# dolocim jakost prvega vrha na 0.834

mol = 'ammonia'
HD = ['pa',.8,1]
peaks = [['p1',.5,1], ['p2', 0, .5]]
dis = []

In [11]:
tr.calibrate(mol, HD, peaks, dis, 70, 620, 1)

In [12]:
fig_num = 1
opis = 'kalibracija za NH3'
plot_name = '%s_%s.png' %(str(fig_num).zfill(2), opis)

plot_path = os.path.join(output_path, plot_name)
plot.show_calibration(tr, save_name=plot_path)

In [37]:
# 2 - analiza prvega TDSja

idx = 3
tr, sv_fn, pl_fn = load_trace(fajli[idx])

HM = {}
HM['ammonia'] = ['a', 'ammonia', ['pa', 0.8, 1]]
HM['water'] = ['w', 'water', ['pw', .9, 1]]

NHM = {}
NHM['nitrogen'] = ['nit', "N2"]

dis = []

t1 = 0
t2 = 1200
t3 = 3000

step = 10

20180508 1802 mid1


In [16]:
tr.deconvolute(HM, NHM, dis, t1, t2, step)

Deconvolutiuon for 1802 mid1 done, duration 9.64327507087 s.


In [19]:
# plot 2 - hiter fit prvega TDSja
fig_num = 2
opis = 'hiter fit prvega TDS-ja'
plot_name = '%s_%s.png' %(str(fig_num).zfill(2), opis)

plot_path = os.path.join(output_path, plot_name)
plot.show_results(tr, save_name=plot_path)

In [38]:
# 3 - fit many times prvega dataseta z enakimi nastavitvami

trs = fmt.fit_many_times(tr, 25, 0.1, [[], ''], HM, NHM, dis, t1, t2, 3)

Doing 25 for 1802 mid1 :
 | 1  | 2  | 3  | 4  | 5  | 6  | 7  | 8  | 9  | 10  | 11  | 12  | 13  | 14  | 15  | 16  | 17  | 18  | 19  | 20  | 21  | 22  | 23  | 24  | 25 deconvolution done, total duration 525.319855658 seconds


In [39]:
fig_num = 3
opis = 'many_times fit prvega TDS-ja'
plot_name = '%s_%s.png' %(str(fig_num).zfill(2), opis)
plot_path = os.path.join(output_path, plot_name)

povp = fmt.povpreci(trs)
fmt.errorbar_results(povp)
plt.savefig(plot_path, dpi=300)
plt.close()

In [2]:
# analiza drugega TDS-ja


idx = 4
tr, sv_fn, pl_fn = load_trace(fajli[idx])

20180508 1802 mid2


In [3]:
HM = {}
HM['ammonia'] = ['a', 'ammonia', 'pa']
HM['water'] = ['w', 'water', 1]

NHM = {}
NHM['nitrogen'] = ['nit', "N2"]

dis = []

t1 = 0
t2 = 3400
t3 = 6000

step = 10

tr, sv_fn, pl_fn = load_trace(fajli[idx], interval=[t2, t3])

20180508 1802 mid2


In [25]:
tr.deconvolute(HM, NHM, dis, t2, t3, step)

Deconvolutiuon for 1802 mid2 done, duration 9.25358679384 s.


In [27]:
# plot 4 - hiter fit drugega TDSja, voda fix na 1
fig_num = 4
opis = 'hiter fit drugega TDS-ja, vodni HD fix na 1'
plot_name = '%s_%s.png' %(str(fig_num).zfill(2), opis)

plot_path = os.path.join(output_path, plot_name)
plot.show_results(tr, save_name=plot_path)

In [4]:
trs = fmt.fit_many_times(tr, 25, 0.1, [[], ''], HM, NHM, dis, 3500, 4000, 3)

Doing 25 for 1802 mid2 :
 | 1  | 2  | 3  | 4  | 5  | 6  | 7  | 8  | 9  | 10  | 11  | 12  | 13  | 14  | 15  | 16  | 17  | 18  | 19  | 20  | 21  | 22  | 23  | 24  | 25 deconvolution done, total duration 163.056985096 seconds


In [7]:
fig_num = 5
opis = 'many_times fit drugega TDS-ja'
plot_name = '%s_%s.png' %(str(fig_num).zfill(2), opis)
plot_path = os.path.join(output_path, plot_name)

povp = fmt.povpreci(trs)
fmt.errorbar_results(povp)
plt.savefig(plot_path, dpi=300)
plt.close()

In [10]:
# Analiza #3 - loading D2 NH3

idx = 5
tr, sv_fn, pl_fn = load_trace(fajli[idx])

20180508 1802 mid3


In [11]:
plot.plot_data(tr)

In [12]:
# loading
t1 = 90
t2 = 1500

In [20]:
HM = {}
HM['hydrogen'] = ['h', 'hydrogen', 'ph']
HM['ammonia'] = ['a', 'ammonia', 'pa']
#HM['NH3_striktno'] = ['NHHH', 'ammonia', 1]
HM['water'] = ['w', 'water', 1]

NHM = {}
NHM['nitrogen'] = ['nit', "N2"]

dis = []

t1 = 90
t2 = 1500
t3 = 4400
t4 = 5400

step = 10

#tr, sv_fn, pl_fn = load_trace(fajli[idx], interval=[t1, t4])

In [21]:
tr.deconvolute(HM, NHM, dis, t1, t2, step)

Deconvolutiuon for 1802 mid3 done, duration 8.33191602286 s.


In [24]:
plot.show_results(tr)

In [23]:
# plot 6 - hiter fit NH3 D2 exposure
fig_num = 6
opis = 'hiter NH3 D2 exposure'
plot_name = '%s_%s.png' %(str(fig_num).zfill(2), opis)

plot_path = os.path.join(output_path, plot_name)
plot.show_results(tr, save_name=plot_path)

In [25]:
# 7 - many times fit istega

trs = fmt.fit_many_times(tr, 25, 0.1, [[], ''], HM, NHM, dis, 130, 1200, 3)

Doing 25 for 1802 mid3 :
 | 1  | 2  | 3  | 4  | 5  | 6  | 7  | 8  | 9  | 10  | 11  | 12  | 13  | 14  | 15  | 16  | 17  | 18  | 19  | 20  | 21  | 22  | 23  | 24  | 25 deconvolution done, total duration 475.029970442 seconds


In [26]:
fig_num = 7
opis = 'many_times fit D2 NH3 exposure'
plot_name = '%s_%s.png' %(str(fig_num).zfill(2), opis)
plot_path = os.path.join(output_path, plot_name)

povp = fmt.povpreci(trs)
fmt.errorbar_results(povp)
plt.savefig(plot_path, dpi=300)
plt.close()

In [30]:
# fit TDSa po NH3 D2

HM = {}
#HM['hydrogen'] = ['h', 'hydrogen', 'ph']
HM['ammonia'] = ['a', 'ammonia', 'pa']
#HM['NH3_striktno'] = ['NHHH', 'ammonia', 1]
HM['water'] = ['w', 'water', 1]

NHM = {}
NHM['nitrogen'] = ['nit', "N2"]

dis = []

t1 = 90
t2 = 1500
t3 = 4400
t4 = 5400

step = 10

tr, sv_fn, pl_fn = load_trace(fajli[idx], interval=[t3, t4])

20180508 1802 mid3


In [31]:
tr.deconvolute(HM, NHM, dis, t3, t4, step)

Deconvolutiuon for 1802 mid3 done, duration 3.36330633532 s.


In [33]:
fig_num = 8
opis = 'hiter fit TDS-a po NH3 D2 exposure'
plot_name = '%s_%s.png' %(str(fig_num).zfill(2), opis)

plot_path = os.path.join(output_path, plot_name)
plot.show_results(tr, save_name=plot_path)

In [34]:
# omejimo fit na t4 = 5200

trs = fmt.fit_many_times(tr, 25, 0.1, [[], ''], HM, NHM, dis, t3, 5200, 3)

Doing 25 for 1802 mid3 :
 | 1  | 2  | 3  | 4  | 5  | 6  | 7  | 8  | 9  | 10  | 11  | 12  | 13  | 14  | 15  | 16  | 17  | 18  | 19  | 20  | 21  | 22  | 23  | 24  | 25 deconvolution done, total duration 206.920641706 seconds


In [36]:
fig_num = 9
opis = 'many_times fit TDS-a po D2 NH3 exposure'
plot_name = '%s_%s.png' %(str(fig_num).zfill(2), opis)
plot_path = os.path.join(output_path, plot_name)

povp = fmt.povpreci(trs)
fmt.errorbar_results(povp)
plt.savefig(plot_path, dpi=300)
plt.close()